# Análise Exploratória de Dados (EDA) e Preprocessamento
Nessa etapa, vamos analisar detalhadamente o conjunto de dados para identificar possíveis problemas, correlações e insights nos dados.

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import ppscore as pps
import emoji
import locale
import warnings

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
pd.set_option('display.float_format', '{:.2f}'.format)
warnings.filterwarnings("ignore")
sns.set(style="whitegrid")

## Read Data

In [ ]:
df = pd.read_csv('./data/raw_data.csv')
df

## Drop Columns
Algumas colunas não serão utilizadas na análise.

In [ ]:
df = df[['DATE', 'SUBJECT', 'SENDER', 'EMAIL', 'DELIVERED', 'OPENS']]
df

## Data Type

In [ ]:
df.info()

In [ ]:
df['DATE'] = pd.to_datetime(df['DATE'].str[:19], format='%Y-%m-%d %H:%M:%S')
df['SUBJECT'] = df.SUBJECT.astype(str)
df['SENDER'] = df.SENDER.astype(str)
df['EMAIL'] = df.EMAIL.astype(str)
df

## Null Values

In [ ]:
df.isnull().sum()

In [ ]:
df['DELIVERED'].fillna(0, inplace=True)
df['OPENS'].fillna(0, inplace=True)
df['EMAIL'].fillna('undefined', inplace=True)

data_dropped = df.dropna(subset=['SUBJECT', 'SENDER'])

data_dropped.isnull().sum()

## Feature Engineering
Adicionar métricas de Open Rate.

In [ ]:
data_dropped = data_dropped.copy()

data_dropped['OPEN_RATE'] = np.where(data_dropped['DELIVERED'] > 0, (data_dropped['OPENS'] / data_dropped['DELIVERED']) * 100, 0)

data_dropped.replace([np.inf, -np.inf], 0, inplace=True)

data_dropped

## Descriptive Statistics
Calcular estatísticas descritivas para DELIVERED, OPENS e as novas métricas.

In [ ]:
descriptive_statistics = data_dropped[['DELIVERED', 'OPENS', 'OPEN_RATE']].describe()
descriptive_statistics

- Também vamos aplicar um filtro para uma quantidade mínima de disparos, afinal campanhas muito pequenas podem atribuir ruídos na análise.
    - Além disso, algumas campanhas pequenas são utilizadas apenas para testes internos, e não devem ser consideradas nesse estudo.
- Vamos considerar apenas campanhas com disparos de >= 100.000

In [ ]:
data_cleaned = data_dropped[(data_dropped['DELIVERED'] >= 100000)].reset_index(drop=True) #(data_dropped['DATE'] >= '2023-01-01') & (data_dropped['DATE'] <= '2023-12-31')

data_cleaned = data_cleaned.copy()

sender_open_rate = data_cleaned.groupby('SENDER')['OPEN_RATE'].mean().reset_index()
sender_open_rate_mapping = sender_open_rate.set_index('SENDER')['OPEN_RATE'].to_dict()
data_cleaned['SENDER_OPEN_RATE'] = data_cleaned['SENDER'].map(sender_open_rate_mapping)
data_cleaned['SENDER_COUNT'] = data_cleaned.groupby('SENDER')['SENDER'].transform('count')

email_open_rate = data_cleaned.groupby('EMAIL')['OPEN_RATE'].mean().reset_index()
email_open_rate_mapping = email_open_rate.set_index('EMAIL')['OPEN_RATE'].to_dict()
data_cleaned['EMAIL_OPEN_RATE'] = data_cleaned['EMAIL'].map(email_open_rate_mapping)
data_cleaned['EMAIL_COUNT'] = data_cleaned.groupby('EMAIL')['EMAIL'].transform('count')

delivered_sum_by_sender = data_cleaned.groupby('SENDER')['DELIVERED'].sum().to_dict()
opens_sum_by_sender = data_cleaned.groupby('SENDER')['OPENS'].sum().to_dict()
data_cleaned['SENDER_SUM_DELIVERED'] = data_cleaned['SENDER'].map(delivered_sum_by_sender).astype(int)
data_cleaned['SENDER_SUM_OPENS'] = data_cleaned['SENDER'].map(opens_sum_by_sender).astype(int)

data_cleaned['HOUR_OF_DAY'] = pd.to_datetime(data_cleaned['DATE']).dt.hour

data_cleaned

In [ ]:
delivered_original = data_cleaned['DELIVERED'].sum()
delivered_formatado = locale.currency(delivered_original, grouping=True, symbol=False)
open_original = data_cleaned['OPENS'].sum()
open_formatado = locale.currency(open_original, grouping=True, symbol=None)
max_date = data_cleaned.DATE.max()
min_date = data_cleaned.DATE.min()

print(f'Quantidade de e-mails enviados: {delivered_formatado}')
print(f'Quantidade de e-mails abertos: {open_formatado}')
print(f'Data Mínima: {min_date}')
print(f'Data Máxima: {max_date}')

In [ ]:
from IPython.display import Markdown

def generate_markdown_stats(descriptive_statistics):
    markdown_text = f"""

- **DELIVERED (E-mails Entregues):**
    - Média: {descriptive_statistics.loc['mean', 'DELIVERED']:.0f}
    - Mediana: {descriptive_statistics.loc['50%', 'DELIVERED']:.0f}
    - Desvio Padrão: {descriptive_statistics.loc['std', 'DELIVERED']:.0f}
    - Mínimo: {descriptive_statistics.loc['min', 'DELIVERED']:.0f}
    - Máximo: {descriptive_statistics.loc['max', 'DELIVERED']:.0f}

- **OPENS (E-mails Abertos):**
    - Média: {descriptive_statistics.loc['mean', 'OPENS']:.0f}
    - Mediana: {descriptive_statistics.loc['50%', 'OPENS']:.0f}
    - Desvio Padrão: {descriptive_statistics.loc['std', 'OPENS']:.0f}
    - Mínimo: {descriptive_statistics.loc['min', 'OPENS']:.0f}
    - Máximo: {descriptive_statistics.loc['max', 'OPENS']:.0f}

- **OPEN_RATE (Taxa de Abertura):**
    - Média: {descriptive_statistics.loc['mean', 'OPEN_RATE']:.2f}%
    - Mediana: {descriptive_statistics.loc['50%', 'OPEN_RATE']:.2f}%
    - Desvio Padrão: {descriptive_statistics.loc['std', 'OPEN_RATE']:.2f}%
    - Mínimo: {descriptive_statistics.loc['min', 'OPEN_RATE']:.2f}%
    - Máximo: {descriptive_statistics.loc['max', 'OPEN_RATE']:.2f}%
    
- **SENDER_OPEN_RATE:**
    - Média: {descriptive_statistics.loc['mean', 'SENDER_OPEN_RATE']:.2f}%
    - Mediana: {descriptive_statistics.loc['50%', 'SENDER_OPEN_RATE']:.2f}%
    - Desvio Padrão: {descriptive_statistics.loc['std', 'SENDER_OPEN_RATE']:.2f}%
    - Mínimo: {descriptive_statistics.loc['min', 'SENDER_OPEN_RATE']:.2f}%
    - Máximo: {descriptive_statistics.loc['max', 'SENDER_OPEN_RATE']:.2f}%
"""
    
    return Markdown(markdown_text)

descriptive_statistics = data_cleaned[['DELIVERED', 'OPENS', 'OPEN_RATE', 'SENDER_OPEN_RATE']].describe()
display(generate_markdown_stats(descriptive_statistics))
descriptive_statistics

## Histogram

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(14, 4))

sns.histplot(data_cleaned['DELIVERED'], bins=30, ax=ax[0], color='skyblue').set_title('DELIVERED')
sns.histplot(data_cleaned['OPENS'], bins=30, ax=ax[1], color='orange').set_title('OPENS')
sns.histplot(data_cleaned['OPEN_RATE'], bins=30, ax=ax[2], color='red').set_title('OPEN_RATE')

plt.tight_layout()
plt.show()

- **DELIVERED:** A distribuição de e-mails entregues é fortemente inclinada à direita, indicando que a maioria das campanhas tem um número relativamente menor de entregas, com algumas campanhas alcançando números muito altos.

- **OPENS e CLICKS:** Ambas as métricas seguem uma distribuição similar à de DELIVERED, com muitas campanhas tendo números menores de aberturas e cliques e poucas alcançando valores extremamente altos.

- **OPEN_RATE:** A distribuição da taxa de abertura mostra uma variedade de comportamentos, com a maioria das campanhas tendo uma taxa de abertura abaixo de 50%, mas com uma distribuição relativamente uniforme.

- **CLICK_RATE:** A taxa de cliques tende a ser mais baixa, com a maioria das campanhas concentrando-se em valores abaixo de 5%, refletindo o desafio em engajar destinatários a ponto de clicarem em um conteúdo.

- **CTOR:** A distribuição da taxa de cliques sobre abertura apresenta uma variação considerável, indicando que, uma vez que o e-mail é aberto, a probabilidade de cliques varia significativamente entre as campanhas.

## Boxplot

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(14, 4))

sns.boxplot(data=data_cleaned, y='OPEN_RATE', ax=ax[0], color='red').set_title('Boxplot of OPEN_RATE')

plt.tight_layout()
plt.show()

- **Boxplot de OPEN_RATE:** A distribuição da taxa de abertura mostra uma mediana abaixo de 30%, com uma gama de valores indicando que algumas campanhas têm taxas de abertura significativamente mais altas. Os outliers acima do terceiro quartil destacam campanhas com desempenho excepcionalmente bom em termos de abertura.

- **Boxplot de CLICK_RATE:** A taxa de cliques tem uma mediana ainda mais baixa, refletindo a dificuldade em motivar os destinatários a interagir com o conteúdo do e-mail. Similar à taxa de abertura, há outliers que indicam campanhas com taxas de cliques excepcionalmente altas.

- **Boxplot de CTOR:** A distribuição da taxa de cliques sobre abertura é ligeiramente mais concentrada do que as outras taxas, mas ainda apresenta outliers, tanto positivos quanto negativos. Isso sugere que, enquanto a maioria das campanhas tem um desempenho relativamente padrão em converter aberturas em cliques, algumas se destacam significativamente, para melhor ou para pior.

## Day of Week
Como as métricas se comportam dependendo do dia da semana.

In [ ]:
data_filtered = data_cleaned.copy()

data_filtered['WEEKDAY'] = data_filtered['DATE'].dt.day_name().str.upper()

weekday_metrics = data_filtered.groupby('WEEKDAY').agg({
    'OPEN_RATE': 'mean',
}).reindex(['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY']).reset_index()

fig_open_rate = px.bar(
    weekday_metrics, 
    x='WEEKDAY', 
    y='OPEN_RATE', 
    title='OPEN_RATE by Day of Week',
    labels={'OPEN_RATE': 'Open Rate (%)', 'WEEKDAY': 'Day of the Week'},
    color='OPEN_RATE', color_continuous_scale='Reds'
)
fig_open_rate.show()

Estes resultados sugerem que o dia da semana pode ter um impacto nas taxas de abertura e cliques das campanhas de e-mail. Portanto, ao planejar o envio de e-mails, considerar o dia da semana pode ajudar a otimizar o desempenho das campanhas.

## Analysis of Open Rate
Analisar especificamente as métricas relacionadas ao Open Rate. Primeiramente precisamos remover as colunas que não vamos ter disponíveis no momento da predição.

In [ ]:
data_selected = data_filtered.drop(['OPENS'], axis=1)
data_selected

## Morphological Features
Vamos nos concentrar nas características morfológicas, procurando por padrões gramaticais e de sintaxe. Ou seja, vamos observar o número de caracteres, o número de palavras, o total de destinatários, o dia da campanha e etc. Isso tudo para Subject e Sender.

*"Considere algumas linhas de assunto com a palavra “grátis”. A palavra “grátis” possui nuances de significado, que influenciam o valor semântico da palavra dependendo do contexto. Além disso, dependendo do país ou da tendência publicitária nacional, as linhas de assunto com essa palavra podem apresentar uma taxa de abertura mais alta. Ou cair em “spam”, seja por clientes de e-mail ou por percepção."*

In [ ]:
def contains_emoji(text):
    for character in text:
        if character in emoji.EMOJI_DATA:
            return 1
    return 0

data_selected['SUBJECT_LENGTH'] = data_selected['SUBJECT'].apply(len)
data_selected['SUBJECT_WORD_COUNT'] = data_selected['SUBJECT'].apply(lambda x: len(x.split()))
data_selected['SUBJECT_SPECIAL_CHARS_COUNT'] = data_selected['SUBJECT'].apply(lambda x: sum(not c.isalnum() for c in x))
data_selected['SUBJECT_NUMBERS_COUNT'] = data_selected['SUBJECT'].apply(lambda x: sum(c.isdigit() for c in x))
data_selected['SUBJECT_HAS_EMOJI'] = data_selected['SUBJECT'].apply(contains_emoji)

data_selected['SENDER_LENGTH'] = data_selected['SENDER'].apply(len)
data_selected['SENDER_WORD_COUNT'] = data_selected['SENDER'].apply(lambda x: len(x.split()))
data_selected['SENDER_SPECIAL_CHARS_COUNT'] = data_selected['SENDER'].apply(lambda x: sum(not c.isalnum() for c in x))
data_selected['SENDER_NUMBERS_COUNT'] = data_selected['SENDER'].apply(lambda x: sum(c.isdigit() for c in x))

data_selected

## Violin Plot Open Rate x Emoji
Este tipo de gráfico é usado para visualizar a distribuição de dados e sua probabilidade de densidade. O aspecto que se assemelha a um violino mostra tanto a caixa de um box plot com uma marca para a mediana, como também a densidade dos dados onde a largura do "violino" representa a frequência.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
ax.set_facecolor('#eaeaf3')
ax.grid(False)

my_pal = {0: "g", 1: "b"}
sns.violinplot(x="SUBJECT_HAS_EMOJI", y="OPEN_RATE", hue="SUBJECT_HAS_EMOJI", data=data_selected, palette=my_pal, legend=False)

ax.set_xlabel("Emoji", fontweight='bold')
ax.set_ylabel("Open Rate", fontweight='bold')

for spine in ax.spines.values():
    spine.set_visible(False)

plt.show()

- **Densidade:** Podemos observar que a densidade das campanhas que tem emoji tem uma concentração maior e mais alta. Indicando que os emojis aumentam a taxa de abertura.

## Line Plot Open Rate x Number of Words
Taxa de abertura por número de palavras do Subject e Sender.

In [ ]:
data_number_words = data_selected.groupby('SUBJECT_WORD_COUNT').agg({'OPEN_RATE': 'mean', 'DATE': 'count'}).rename(columns={'DATE': 'COUNT'}).reset_index().sort_values(by='SUBJECT_WORD_COUNT')

fig, ax = plt.subplots(figsize=(16, 8))
ax.set_facecolor('#eaeaf3')
ax.grid(False)

ax.plot(data_number_words['SUBJECT_WORD_COUNT'], data_number_words['OPEN_RATE'], color='blue')
ax.fill_between(data_number_words['SUBJECT_WORD_COUNT'], data_number_words['OPEN_RATE'], color='skyblue')

for spine in ax.spines.values():
    spine.set_visible(False)

ax.set_xlabel("Number of Words", fontweight='bold')
ax.set_ylabel("Open Rate", fontweight='bold')

ax.set_xlim(data_number_words['SUBJECT_WORD_COUNT'].min(), data_number_words['SUBJECT_WORD_COUNT'].max())

plt.show()

## Line Plot Open Rate x Lenght

In [ ]:
data_number_words = data_selected.groupby('SUBJECT_LENGTH').agg({'OPEN_RATE': 'mean', 'DATE': 'count'}).rename(columns={'DATE': 'COUNT'}).reset_index().sort_values(by='SUBJECT_LENGTH')

fig, ax = plt.subplots(figsize=(16, 8))
ax.set_facecolor('#eaeaf3')
ax.grid(False)

ax.plot(data_number_words['SUBJECT_LENGTH'], data_number_words['OPEN_RATE'], color='blue')
ax.fill_between(data_number_words['SUBJECT_LENGTH'], data_number_words['OPEN_RATE'], color='skyblue')

for spine in ax.spines.values():
    spine.set_visible(False)

ax.set_xlabel("Subject Length", fontweight='bold')
ax.set_ylabel("Open Rate", fontweight='bold')

ax.set_xlim(data_number_words['SUBJECT_LENGTH'].min(), data_number_words['SUBJECT_LENGTH'].max())

plt.show()

Cada Sender para ter uma performance diferente de Open Rate

## ANOVA Open Rate x Sender
Verificar se existem diferenças estatisticamente significativas nas taxas de abertura médias (OPEN_RATE) entre diferentes remetentes (SENDER).

In [ ]:
import scipy.stats as stats


groups = data_selected.groupby('SENDER')['OPEN_RATE'].apply(list)

f_value, p_value = stats.f_oneway(*groups)
print(f"F-value: {f_value}, P-value: {p_value}")

- p-value muito baixo (2.1724430416e-314), que é praticamente zero, indica que há uma diferença estatisticamente significativa nas taxas de abertura médias entre os diferentes remetentes. Isso significa que o remetente (SENDER) tem um efeito significativo na taxa de abertura (OPEN_RATE).

## Correlation Matrix
Utilizando a correlação de Pearson para as variáveis numéricas.

In [ ]:
data_selected.drop(columns=['DATE', 'SUBJECT', 'SENDER', 'WEEKDAY', 'EMAIL'])

In [ ]:
corr_matrix = data_selected.drop(columns=['DATE', 'SUBJECT', 'SENDER', 'WEEKDAY', 'EMAIL']).corr()

plt.figure(figsize=(16, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Matriz de Correlação')
plt.show()

Análise inicial:

- **SENDER_OPEN_RATE (0.62):** É a correlação forte mais relevante com o OPEN_RATE, demostrando ser a feature mais importante nos dados.

- **DELIVERED (-0.18):** Existe uma correlação negativa moderada com o OPEN_RATE. Isso sugere que, à medida que o número de e-mails entregues aumenta, o OPEN_RATE tende a diminuir levemente. Isso pode ser devido ao fato de campanhas maiores alcançarem um público mais amplo e diversificado, possivelmente menos engajado.

- **SUBJECT_LENGTH (0.04) e SUBJECT_WORD_COUNT (-0.01):** As características do assunto, como o comprimento e a contagem de palavras, têm uma correlação muito fraca com o OPEN_RATE, indicando que esses fatores, por si só, não têm um impacto significativo na taxa de abertura.

- **SUBJECT_SPECIAL_CHARS_COUNT (0.18) e SUBJECT_NUMBERS_COUNT (0.11):** O número de caracteres especiais e números no assunto mostra uma correlação positiva fraca a moderada com o OPEN_RATE. Isso pode indicar que o uso de caracteres especiais e números (que podem sugerir urgência ou ofertas específicas) pode aumentar ligeiramente a taxa de abertura.

- **SUBJECT_HAS_EMOJI (0.19):** A presença de emojis no assunto tem a correlação mais forte (ainda que moderada) com o OPEN_RATE entre as características analisadas. Isso sugere que assuntos com emojis podem ser mais atraentes e gerar maiores taxas de abertura.

- **SENDER_LENGTH (-0.04), SENDER_WORD_COUNT (0.05), SENDER_SPECIAL_CHARS_COUNT (0.05), e SENDER_NUMBERS_COUNT (-0.06):** As características do remetente têm correlações muito fracas com o OPEN_RATE, indicando que esses fatores têm pouca influência direta na probabilidade de os e-mails serem abertos.

Logo, podemos concluir que:

1. A presença de emojis no assunto se destaca como um fator que potencialmente aumenta o OPEN_RATE, o que pode ser explorado para tornar as campanhas mais eficazes.

2. O número de caracteres especiais no assunto também está positivamente correlacionado com o OPEN_RATE, sugerindo que o design criativo do assunto pode engajar mais os destinatários.

3. A correlação negativa entre DELIVERED e OPEN_RATE reforça a ideia de que o engajamento pode ser desafiador em campanhas de grande escala.

**As correlações fracas reforçam ainda mais a ideia de que a taxa de abertura dos e-mails está diretamente ligada com o conteúdo do Subject/Sender, a semântica da frase, e não características morfológicas do texto.**

## Predictive Power Score
Diferentemente da correlação de Pearson, que mede apenas a relação linear e é limitada a variáveis numéricas, o PPS pode ser usado para descobrir e interpretar relações lineares e não lineares entre variáveis numéricas e categóricas.

O PPS é calculado treinando um modelo de machine learning (como uma árvore de decisão) para prever uma variável com base em outra. O desempenho do modelo é então comparado ao de um modelo naif (baseline) que sempre prevê o valor mais comum da variável alvo. O PPS é uma pontuação que varia de 0 a 1, onde 0 significa que a variável preditora não tem capacidade de prever a variável alvo melhor do que o modelo naif, e 1 significa que a variável preditora pode prever a variável alvo perfeitamente.

In [ ]:
def heatmap(df):
    plt.figure(figsize=(16,8))
    df = df[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore')
    ax = sns.heatmap(df, vmin=0, vmax=1, cmap="coolwarm", linewidths=0.5, annot=True)
    ax.set_title("PPS matrix")
    ax.set_xlabel("feature")
    ax.set_ylabel("target")
    plt.show()

matrix = pps.matrix(data_selected)
heatmap(matrix)

## Feature Engineering

- MORNING (Manhã): Tipicamente considerado das 6h às 12h.
- AFTERNOON (Tarde): Geralmente das 12h às 18h.
- EVENING (Final de Tarde/Início da Noite): Normalmente das 18h às 24h.
- NIGHT (Madrugada): Normalmente das 0h às 6h.

In [ ]:
def categorize_hour_of_day(hour):
    if 0 <= hour < 6:
        return 'NIGHT'
    elif 6 <= hour < 12:
        return 'MORNING'
    elif 12 <= hour < 18:
        return 'AFTERNOON'
    else:
        return 'EVENING'

data_to_save = data_filtered[['SUBJECT', 'SENDER', 'EMAIL', 'DELIVERED', 'OPEN_RATE', 'HOUR_OF_DAY', 'WEEKDAY']]

max_delivered = data_to_save['DELIVERED'].max()
data_to_save['DELIVERED_SCALED'] = data_to_save['DELIVERED'] / max_delivered

data_to_save['TIME_OF_DAY'] = data_to_save['HOUR_OF_DAY'].apply(categorize_hour_of_day)

average_open_rate_hour = data_to_save.groupby('HOUR_OF_DAY')['OPEN_RATE'].mean().reset_index()
average_open_rate_hour.columns = ['HOUR_OF_DAY', 'HOUR_OPEN_RATE']

average_open_rate_weekday = data_to_save.groupby('WEEKDAY')['OPEN_RATE'].mean().reset_index()
average_open_rate_weekday.columns = ['WEEKDAY', 'WEEKDAY_OPEN_RATE']

average_open_rate_timeofday = data_to_save.groupby('TIME_OF_DAY')['OPEN_RATE'].mean().reset_index()
average_open_rate_timeofday.columns = ['TIME_OF_DAY', 'TIME_OF_DAY_OPEN_RATE']

average_open_rate_sender = data_to_save.groupby('SENDER')['OPEN_RATE'].mean().reset_index()
average_open_rate_sender.columns = ['SENDER', 'SENDER_OPEN_RATE']

average_open_rate_email = data_to_save.groupby('EMAIL')['OPEN_RATE'].mean().reset_index()
average_open_rate_email.columns = ['EMAIL', 'EMAIL_OPEN_RATE']

data_to_save = data_to_save.merge(average_open_rate_hour, on='HOUR_OF_DAY', how='left')
data_to_save = data_to_save.merge(average_open_rate_weekday, on='WEEKDAY', how='left')
data_to_save = data_to_save.merge(average_open_rate_timeofday, on='TIME_OF_DAY', how='left')
data_to_save = data_to_save.merge(average_open_rate_sender, on='SENDER', how='left')
data_to_save = data_to_save.merge(average_open_rate_email, on='EMAIL', how='left')

data_to_save

## Save Data

In [ ]:
data_to_save.to_csv('./data/data_preprocessed.csv', index=False)